In [1]:
import json
import pandas as pd
import sqlite3
import os
from datetime import datetime
import os.path
import time

In [5]:
print('iniciando ingestão!')
time.sleep(3)
file_col_json = '/workspaces/api-geral/ING/app/src/layouts/cep/tabela_cep.json'
file_tb_json = '/workspaces/api-geral/ING/app/src/layouts/cep/estrutura.json'
bckp_path = '/workspaces/api-geral/ING/app/src/bckp/'
landing = '/workspaces/api-geral/ING/app/src/data/landing/'
word_key = 'resultado'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

ultimo_timestamp = None
caminho_arquivo = None

print('carregando variaveis json')
time.sleep(3)
# Ler configurações das colunas
with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')

time.sleep(3)
# Ler configurações da tabela
with open(f'{file_tb_json}', 'r') as f:
    config_tabela = json.load(f)
    print('lendo estrutura tabela json')

print('carregando variaveis do arquivo de ingestão')
time.sleep(3)

print('realizando iteração')
print('buscando o ultimo arquivo ingerido de acordo com o timestamp')
time.sleep(3)
for root, dirs, files in os.walk(landing):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                print('arquivo encontrado!')
                print('realizando checking do arquivo.')
                time.sleep(3)
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.json'}:
                        print(f'file extension json, ok! extension: {file_extension}')
                    else:
                        print('extension json error, not found')
                    print(caminho_arquivo)

print('carregando arquivo...')
time.sleep(3)
# Carregar dados
arquivo_dados = caminho_arquivo
if arquivo_dados.endswith('.txt'):
    dados = pd.read_csv(arquivo_dados, delimiter=';', header=None, names=[col['nome'] for col in config_colunas['colunas']])
elif arquivo_dados.endswith('.csv'):
    dados = pd.read_csv(arquivo_dados, header=None, names=[col['nome'] for col in config_colunas['colunas']])
elif arquivo_dados.endswith('.bin'):
    # Lógica para carregar dados de um arquivo binário
    pass
else:
    raise ValueError("Formato de arquivo não suportado")

print('convertendo tipos de dados')
time.sleep(3)
# Conversão de tipos de dados
for col in config_colunas['colunas']:
    if col['tipo'] == 'int':
        dados[col['nome']] = dados[col['nome']].astype(int)
    elif col['tipo'] == 'float':
        dados[col['nome']] = dados[col['nome']].astype(float)
    elif col['tipo'] == 'data':
        dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

print('conctando no banco de dados')
# Conectar ao banco de dados
banco = '/workspaces/api-geral/ING/database/banco_cep_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

# Criar tabela com base na estrutura definida no arquivo JSON
print('Criando tabela no banco de dados...')
sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_tabela['tabela']} ("
for col in config_colunas['colunas']:
    sql_create_table += f"{col['nome']} {col['tipo']}, "
sql_create_table = sql_create_table[:-2] + ")"
cursor.execute(sql_create_table)

# Inserir dados na tabela
print('Inserindo dados na tabela...')
for index, row in dados.iterrows():
    mapped_values = []
    for col in config_colunas['colunas']:
        mapped_values.append(row.get(col['nome'], None))
    print("Valores mapeados:", mapped_values)
    sql = f"INSERT INTO {config_tabela['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
    print("SQL:", sql)
    cursor.execute(sql, mapped_values)

# Confirmar as alterações e fechar a conexão
print('Commit e fechamento da conexão...')
conn.commit()
conn.close()

print("Ingestão de dados concluída com sucesso!")


iniciando ingestão!
carregando variaveis json
lendo estrutura de coluna json
lendo estrutura tabela json
carregando variaveis do arquivo de ingestão
realizando iteração
buscando o ultimo arquivo ingerido de acordo com o timestamp
arquivo encontrado!
realizando checking do arquivo.
True
extension json error, not found
/workspaces/api-geral/ING/app/src/data/landing/resultado_concat_20240513_162447.txt
carregando arquivo...
convertendo tipos de dados
conctando no banco de dados
Criando tabela no banco de dados...
Inserindo dados na tabela...
Valores mapeados: ['logradouro', 'bairro', 'localidade', 'uf', 'ibge', 'gia', 'ddd', 'siafi']
SQL: INSERT INTO tb_cep_brnz (logradouro, bairro, localidade, uf, ibge, gia, ddd, siafi) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
Valores mapeados: ['Rua Mairiporã', 'Mantiqueira', 'Mairiporã', 'SP', '3528502', '4339', '11', '6671']
SQL: INSERT INTO tb_cep_brnz (logradouro, bairro, localidade, uf, ibge, gia, ddd, siafi) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
Valores mapeados

In [6]:
# Conectar ao banco de dados
banco = '/workspaces/api-geral/ING/database/banco_cep_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

tabela = 'tb_cep_brnz'
query = f'''
select * from {tabela}
'''
cursor.execute(query)
resultados = cursor.fetchall()
print('Resultado:')
for linha in resultados:
    print(linha)    

Resultado:
('logradouro', 'bairro', 'localidade', 'uf', 'ibge', 'gia', 'ddd', 'siafi')
('Rua Mairiporã', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua JasminsJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua OrquideasJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua GirassolJd Mairipora II', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua das RosasJd Mairipora II', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua AntúriosJd Mairipora II', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Rua dos CravosJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda LíriosJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda TulipasJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda GiestasJd Mairipora I', 'Rio Abaixo', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('Alameda GirassolJd 

In [4]:
query = 'drop table tb_cep_brnz'
cursor.execute(query)
print('tabela excluida')
conn.commit()
conn.close()

OperationalError: no such table: tb_cep_brnz